In [1]:
import math
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
from vocab import tokens

In [23]:
class GPT(nn.Module):
    def __init__(self, tokens_list):
        super().__init__()
        self.tokens_list = tokens_list
        self.max_tokens = 10
        self.context_length = 64
        self.vocab_size = 200
        self.n_layers = 1
        self.n_heads = 1
        self.n_embd = 16

        self.wte = nn.Embedding(self.vocab_size, self.n_embd)     # Word Token Embedding
        self.wpe = nn.Embedding(self.context_length, self.n_embd) # Positional Embedding

        self.fc1 = nn.Linear(self.n_embd, 2 * self.n_embd)
        self.gelu = nn.GELU(approximate='tanh')  # GELU usada no GPT
        self.fc2 = nn.Linear(2 * self.n_embd, self.n_embd)

        self.ln = nn.LayerNorm(self.n_embd)

        self.qkv_proj = nn.Linear(self.n_embd, 3 * self.n_embd)  # Projeção para Q, K, V
        self.out_proj = nn.Linear(self.n_embd, self.n_embd)      # Projeção final
        self.head_dim = self.n_embd // self.n_heads              # Dimensão de cada cabeça
        
        self.register_buffer(
            "mask",
            torch.tril(torch.ones(self.context_length, self.context_length))
                .view(1, 1, self.context_length, self.context_length)
        )
        
        self.final_ln = nn.LayerNorm(self.n_embd)  # LayerNorm final
        self.lm_head = nn.Linear(self.n_embd, self.vocab_size)  # Mapeia para o vocabulário

        assert self.n_embd % self.n_heads == 0, "n_embd deve ser divisível por n_heads"

    def mlp(self, x):
        # x: (T, C)
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.fc2(x)
        return x
    
    def layer_norm(self, x):
        # x: (T, C)
        return self.ln(x)
    
    def self_attention(self, x):
        # x: (T, C) → sequência, embedding
        T, C = x.size()

        # Projeta para Q, K, V
        qkv = self.qkv_proj(x)  # (T, 3*C)
        q, k, v = qkv.chunk(3, dim=1)  # Cada um (T, C)

        # Separa em múltiplas heads
        #q = q.view(T, self.n_heads, self.head_dim).transpose(0, 1)  # (nh, T, hd)
        #k = k.view(T, self.n_heads, self.head_dim).transpose(0, 1)
        #v = v.view(T, self.n_heads, self.head_dim).transpose(0, 1)

        # Produto escalar entre Q e Kᵀ
        att = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)  # (nh, T, T)

        # Aplicar máscara causal
        att = att.masked_fill(self.mask[:, :, :T, :T] == 0, float('-inf'))

        # Normaliza com Softmax
        att = torch.softmax(att, dim=-1)

        # Atenção aplicada sobre V
        y = att @ v  # (nh, T, hd)

        # Junta as heads
        #y = y.transpose(0, 1).contiguous().view(T, C)  # (T, C)

        # Projeção final
        y = self.out_proj(y)
        return y
    

    def tokens_idx(self, tokens_chosen):
        self.tokens_vocab = {token: idx for idx, token in enumerate(tokens)}
        self.idx_to_token = {idx: token for token, idx in self.tokens_vocab.items()}

        # Pegar os índices correspondentes
        indices = [self.tokens_vocab[token] for token in tokens_chosen]

        # Converter para tensor, se quiser passar ao modelo
        self.indices_tensor = torch.tensor(indices, dtype=torch.long)

        #print("Tokens escolhidos:", tokens_escolhidos)
        #print("Índices encontrados:", indices)
        #print("Tensor de índices:", indices_tensor)
        #return indices_tensor

    def forward(self):
        """
        Executa a passagem para frente (forward pass) do modelo.
        Usa os índices de tokens armazenados em self.indices_tensor.
        """

        # === Entrada: índices dos tokens ===
        idx = self.indices_tensor  # Tensor de índices (T,)
        T = idx.size(0)             # Número de tokens
        #print(f"Tamanho da sequência (T): {T}")
        #print(f"Shape de idx: {idx.shape}")  # (T,)

        # === Embedding dos tokens ===
        tok_emb = self.wte(idx)  # Embedding dos tokens (T, n_embd)
        #print("\nEmbedding dos Tokens (tok_emb):")
        #print(f"Shape: {tok_emb.shape}")  # (T, n_embd)
        #print(tok_emb)

        # === Embedding das posições ===
        positions = torch.arange(T, dtype=torch.long, device=idx.device)  # (T,)
        pos_emb = self.wpe(positions)  # Embedding das posições (T, n_embd)
        #print("\nEmbedding das Posições (pos_emb):")
        #print(f"Shape: {pos_emb.shape}")  # (T, n_embd)
        #print(pos_emb)

        # === Soma dos embeddings ===
        x = tok_emb + pos_emb  # Combinação token + posição (T, n_embd)
        #print("\nSoma dos Embeddings (x = tok_emb + pos_emb):")
        #print(f"Shape: {x.shape}")  # (T, n_embd)
        #print(x)

        # === Normalização antes da atenção ===
        x_norm = self.layer_norm(x)  # (T, n_embd)
        #print("\nLayerNorm aplicado na soma (x_norm):")
        #print(f"Shape: {x_norm.shape}")  # (T, n_embd)
        #print(x_norm)

        # === Atenção (Self-Attention) ===
        attn_out = self.self_attention(x_norm)  # (T, n_embd)
        #print("\nSaída da Self-Attention (attn_out):")
        #print(f"Shape: {attn_out.shape}")  # (T, n_embd)
        #print(attn_out)

        # === Soma residual (após atenção) ===
        x = x + attn_out  # (T, n_embd)
        #print("\nSoma Residual após Atenção (x):")
        #print(f"Shape: {x.shape}")  # (T, n_embd)
        #print(x)

        # === Segundo LayerNorm (antes do MLP) ===
        x_norm = self.layer_norm(x)  # (T, n_embd)
        #print("\nSegundo LayerNorm antes do MLP (x_norm):")
        #print(f"Shape: {x_norm.shape}")  # (T, n_embd)
        #print(x_norm)

        # === Passagem pelo MLP ===
        mlp_out = self.mlp(x_norm)  # (T, n_embd)
        #print("\nSaída do MLP (mlp_out):")
        #print(f"Shape: {mlp_out.shape}")  # (T, n_embd)
        #print(mlp_out)

        # === Soma Residual após o MLP ===
        x = x + mlp_out  # (T, n_embd)
        #print("\nSoma Residual após o MLP (x):")
        #print(f"Shape: {x.shape}")  # (T, n_embd)
        #print(x)

        return x[0,0,-1,:]
    
    def predict_logits(self, x):
        """
        Recebe a saída do forward (T, n_embd),
        aplica LayerNorm final e gera logits para o vocabulário (T, vocab_size).
        """
        x = self.final_ln(x)  # Normaliza novamente
        logits = self.lm_head(x)  # Projeta para vocab_size
        #print("\nLogits (após LayerNorm final):")
        #print(f"Shape: {logits.shape}")  # (T, vocab_size)
        #print(logits)
        return logits

    def predict_next_token(self):
        self.tokens_idx(self.tokens_list)
        res = self.forward()
        logits = self.predict_logits(res)
        probs = torch.softmax(logits, dim=-1)  # (T, vocab_size)
        next_token_idx = torch.multinomial(probs, num_samples=1)
        next_token_idx = int(next_token_idx[0])

        return self.idx_to_token[next_token_idx]
    
    def predict_all_sentence(self):
        """
        Gera tokens autoregressivamente até atingir max_tokens,
        usando predict_next_token e imprimindo a frase completa a cada passo.
        """
        # Itera até o limite de tokens definidos em self.max_tokens
        for step in range(self.max_tokens):
            # Prediz o próximo token com base no estado atual
            next_token = self.predict_next_token()
            # Adiciona o token gerado à lista de tokens
            self.tokens_list.append(next_token)
            # Constrói a frase atualizada e imprime
            sentence = " ".join(self.tokens_list)
            print(f"Passo {step+1}: {sentence}")
        # Retorna a lista completa de tokens gerados
        return self.tokens_list

In [24]:
tokens_list = ["o", "gato", "pequeno"]
model = GPT(tokens_list)

In [25]:
model.predict_next_token()

'dinheiro'

In [26]:
model.predict_all_sentence()

Passo 1: o gato pequeno porta
Passo 2: o gato pequeno porta trinta
Passo 3: o gato pequeno porta trinta dinheiro
Passo 4: o gato pequeno porta trinta dinheiro dez
Passo 5: o gato pequeno porta trinta dinheiro dez uns
Passo 6: o gato pequeno porta trinta dinheiro dez uns dormir
Passo 7: o gato pequeno porta trinta dinheiro dez uns dormir não
Passo 8: o gato pequeno porta trinta dinheiro dez uns dormir não escola
Passo 9: o gato pequeno porta trinta dinheiro dez uns dormir não escola vocês
Passo 10: o gato pequeno porta trinta dinheiro dez uns dormir não escola vocês cinco


['o',
 'gato',
 'pequeno',
 'porta',
 'trinta',
 'dinheiro',
 'dez',
 'uns',
 'dormir',
 'não',
 'escola',
 'vocês',
 'cinco']